In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

In [4]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_dict = {i: months[i-1] for i in range(0, 13)}

In [ ]:
pd_data = pd.read_csv('pd_data_clean.csv')
pd_data = pd_data.dropna(subset=['Location'])
strs = []
for i in range(len(pd_data)):
    month = (pd_data.iloc[i]['MONTH'].astype(int))
    year = pd_data.iloc[i]['YEAR']
    strs.append(f'{year} {month}')
pd_data['DATE'] = strs
pd_data.head()

In [ ]:
def get_point(loc):
    loc = str(loc)
    loc = loc.strip("()").split(',')
    lat, long = map(float, loc)
    point = Point(long, lat)
    return point

In [ ]:
pd_data['Coordinates'] = pd_data['Location'].apply(get_point)
gdf = gpd.GeoDataFrame(pd_data, geometry='Coordinates')

In [ ]:
boston_bg = gpd.read_file('2020_Census_Block_Groups_in_Boston.shp')

In [ ]:
gdf = gdf.set_crs("EPSG:4326")
#gdf = gdf.to_crs('EPSG:2249')
boston_bg = boston_bg.set_crs('EPSG:2249', allow_override=True)
boston_bg = boston_bg.to_crs('EPSG:4326')

In [ ]:
print(gdf.crs)  # Check CRS of the GeoDataFrame
print(boston_bg.crs)

In [ ]:
joined_gdf = gpd.sjoin(gdf, boston_bg, how='right', predicate='within')

In [ ]:
joined_df = joined_gdf.set_geometry('geometry', crs = 'EPSG:4326')

In [ ]:
to_map = gpd.GeoDataFrame(joined_gdf.groupby(['DATE', 'geometry'], as_index = False)['SERIOUS'].sum())
to_map = to_map.set_geometry('geometry', crs = 'EPSG:4326')

In [ ]:
to_map['DATE'] = pd.to_datetime(to_map['DATE'], format='%Y %m')
#to_map['DATE'] = to_map['DATE'].dt.strftime('%Y-%m')

In [ ]:
import plotly.express as px
import json

In [ ]:
minx, miny, maxx, maxy = to_map.total_bounds

In [ ]:
minx= minx-10
miny=miny-10
maxx=maxx+10
maxy=maxy+10

In [ ]:
fig = px.choropleth_mapbox(to_map, geojson = to_map.geometry, color = "SERIOUS", 
                           mapbox_style="carto-positron", animation_frame = 'DATE')

In [ ]:
fig.update_layout(
    mapbox_bounds = {'west': minx, 'east': maxx, 'south': miny, 'north':maxy}
    mapbox_center={"lat": 42.3399, "lon": -71.0899},  
    mapbox_zoom=3 
)

In [ ]:
fig.show()

In [ ]:
fig.save("test_chloro.html")